### Importing libraries

In [1]:
import re
import glob
import os
import datetime
from datetime import date
from nltk.tokenize import word_tokenize
from collections import Counter
import pandas as pd
import numpy as np
import rpy2.rinterface
from functools import partial
#from rpy2.robjects.packages import importr, data
#from rpy2.robjects import pandas2ri
import math

### Regular expressions and month dictionaries

In [2]:
head_judge_it = re.compile('Il\sPRESIDENTE\,*\n', re.I)
head_judge_fr = re.compile('L[aen]\sPRÉSIDENT\,*\n', re.I)
verdict_lines_fr = re.compile('ces\sMotifs\:', re.I)
verdict_lines_short_fr = re.compile('ces\sMo')
verdict_lines_it = re.compile('questi\sMotivi\:', re.I)
verdict_lines_short_it = re.compile('esti\sMo')

parties_it = re.compile('^contro$', re.I)
parties_fr = re.compile('^contre$', re.I)


period_new_line = re.compile('\.\n')
lawyer = re.compile('Av+\.?')
courts_it = re.compile('CORTE')
courts_fr = re.compile('COUR') 
numbers = re.compile('\d')
courts_end_line_it = re.compile('La\sCorte\,?\n?')
courts_end_line_fr = re.compile('La\sCour\,?\n?')
tribunal_end_line_fr = re.compile('Le\sTribunal\sCivil\,?\n?')
other_courts_end_line = re.compile('La\sCoûte\,?\n?')
next_line = re.compile('\,\n')

date_regex = re.compile('\s[1]([8]|[9])(\.|[0-9])(\.|[0-9])')
city_date = re.compile('Ale[sx]')


new_case_it = re.compile('SOMMARIO\.?\n?')
new_case_fr = re.compile('SOMMAIRE\.?\n?')
new_case_2 = re.compile('\s[1]([8]|[9])(\.|[0-9])(\.|[0-9])\sPr')
new_case_3 = re.compile('\s[1]([8]|[9])(\.|[0-9])(\.|[0-9])\)')


court_breakers_it = re.compile('CORTE\sD\'APPELLO')
court_breakers_fr = re.compile('COUR\sD\'APPEL')

cities_Alexandria_fr = re.compile('Tribunal\sd\'Alexandrie', re.I)
cities_Alexandria_it = re.compile('di\sAlessandria', re.I)
cities_Cairo_fr = re.compile('Tribunal\sm*i*x*t*e*\sdu\sCaire', re.I)
cities_Cairo_it = re.compile('di\s[Cc]airo', re.I)
cities_Mansourah_fr = re.compile('du\sMansourah*', re.I)
cities_Mansourah_it = re.compile('di\sMansourah*', re.I)

ital2int = {'gennaio': 1, 'gennajo' : 1, 'febbraio': 2, 'marzo': 3, 'aprile': 4, 'maggio': 5, 'giugno': 6, 
            'luglio': 7, 'agosto': 8, 'settembre': 9, 'ottobre': 10, 'novembre': 11, 'dicembre': 12, 
            'uecembrc': 12, 'decembre': 12}
french2int = {'janvier': 1, 'janvcr': 1, 'février': 2, 'févvrier': 2, 'fevrier': 2, 'févriar': 2, 
              'mars': 3, 'avril': 4, 'avrll': 4, 'avr1l__': 4, 'août': 4, 'mai': 5, 'juin': 6, 
                'juillet': 7, 'aout': 8, 'septembre': 9, 'octobre': 10, 'novembre': 11, 
                'décembre': 12, 'décelnbre': 12, 'décemhre': 12, 'déceiubre': 12}

### Compiling file directories

In [3]:
dir_list = ["/Users/simabiondi/Downloads/35112103258085","/Users/simabiondi/Downloads/35112103258077", 
            "/Users/simabiondi/Downloads/35112103258093", "/Users/simabiondi/Downloads/35112103258101", 
            "/Users/simabiondi/Downloads/35112103258119", "/Users/simabiondi/Downloads/35112103258127", 
            "/Users/simabiondi/Downloads/35112103258135", "/Users/simabiondi/Downloads/35112103258143", 
            "/Users/simabiondi/Downloads/35112103258150", "/Users/simabiondi/Downloads/35112103258168", 
            "/Users/simabiondi/Downloads/35112103258176", "/Users/simabiondi/Downloads/35112103258184", 
            "/Users/simabiondi/Downloads/35112103258192", "/Users/simabiondi/Downloads/35112103258200", 
            "/Users/simabiondi/Downloads/35112103258218", "/Users/simabiondi/Downloads/35112103258226", 
            "/Users/simabiondi/Downloads/35112105355871", "/Users/simabiondi/Downloads/hl0pps", 
            "/Users/simabiondi/Downloads/hl0ppt", "/Users/simabiondi/Downloads/hl0ppu", 
            "/Users/simabiondi/Downloads/hl0ppv", "/Users/simabiondi/Downloads/hl0ppw", 
            "/Users/simabiondi/Downloads/hl0ppx", "/Users/simabiondi/Downloads/hl0ppy", 
            "/Users/simabiondi/Downloads/hl0pqg", "/Users/simabiondi/Downloads/hl0pqh", 
            "/Users/simabiondi/Downloads/hl0pqi", "/Users/simabiondi/Downloads/hl0pqj", 
            "/Users/simabiondi/Downloads/hl0pqk", "/Users/simabiondi/Downloads/hl0pr2", 
            "/Users/simabiondi/Downloads/hl0pr4", "/Users/simabiondi/Downloads/hl0pr5", 
            "/Users/simabiondi/Downloads/hl0pr6", "/Users/simabiondi/Downloads/hl0pr7", 
            "/Users/simabiondi/Downloads/hl0prq", "/Users/simabiondi/Downloads/hl0prv"]
f_dat = []
for d in dir_list:
    os.chdir(d)
    txt_files = (glob.glob('00000*.txt'))
    for f in txt_files:       
        fin = open(f, encoding = "utf-8")
        f_dat.append(fin.readlines())

### Separating and identifying individual cases

In [4]:
c_dat = []
temp = []
for t in f_dat:
    for line in t:
        if(new_case_2.search(line) or new_case_3.search(line)):
            temp.append(line)
            c_dat.append(temp)
            temp = []        
        elif((new_case_it.match(line) or new_case_fr.match(line)) and len(temp) > 20): 
            c_dat.append(temp)
            temp = []
        elif(not court_breakers_fr.search(line) or not court_breakers_it.search(line)):
            temp.append(line)  

### Functions

#### Name of the head judge

In [5]:
def head_judge_fun(line):
    pres_index = case.index(str(line))
    hj = ""
    try: 
        hj = (case[pres_index + 1].strip())
    except: 
        hj = np.nan
    return hj

#### Case's date

In [6]:
def date_and_lang_fun(line):
    indx = case.index(str(line))
    tokens = word_tokenize(re.sub(r'[^\w\s]','', (case[indx-1]).strip()))
    date_tokens = tokens[len(tokens)-3:len(tokens)]
    return_val = []
    try: 
        year = int(date_tokens[2])
        day = int(date_tokens[0])
        mo = 0
        if((date_tokens[1]).lower() in ital2int):
            mo = ital2int[date_tokens[1].lower()]
            language = "italian"
        elif((date_tokens[1]).lower() in french2int):
            mo = french2int[date_tokens[1].lower()]
            language = "french"
        else:
            print(date_tokens)
            
        return_val = [day, mo, year, language]
    except: 
        return_val = [np.nan, np.nan, np.nan, ""]

        
    return return_val

#### Litigants and their lawyers

In [7]:
def parties_fun(line):
   
    # Isolating plaintiff's lines and plaintiff's lawyer's name
    p_return = plaintiff_fun(line)

    # Isolating defendant's lines and defendant's lawyer's name
    d_return = defendants_fun(line)
            
    return_val = [p_return, d_return]
    return return_val

#### Plaintiffs and their lawyers

In [8]:
def plaintiff_fun(line):
    con_index = case.index(str(line))
    
    #Getting all the plaintiff lines
    p_lines = case[con_index-3: con_index]
    p_lines = [item.strip() for item in p_lines]
    
    #Dividing between plaintifs and lawyers
    p_lawyer_names = []
    plaintiffs = []
    for item in p_lines:
        if(lawyer.search(item)):
            p_lawyer_tokens = item.split()
            for name in p_lawyer_tokens:
                    if(lawyer.search(name)):
                        name_idx = p_lawyer_tokens.index(name)
                        if(name_idx < (len(p_lawyer_tokens) - 1)):
                            p_lawyer_names.append(p_lawyer_tokens[name_idx + 1])
                            try:
                                plaintiffs.append(p_lawyer_tokens[:name_idx-1])
                            except:
                                plaintiffs.append(p_lines[item-1])
        else:
            plaintiffs.append(item)
    return [p_lawyer_names, plaintiffs]                        

#### Defendants and their lawyers

In [9]:
def defendants_fun(line):
    con_index = case.index(str(line))
    d_lines = []
    #Getting all the defendants lines
    for i in range(1, 6):
        try:
            l = case[con_index+i]
            if(not next_line.search(l) and not courts_end_line_it.search(l) and not courts_end_line_fr.search(l)):
                d_lines.append(l.strip())
            if(courts_end_line_fr.search(l) or courts_end_line_it.search(l)):
                break
        except:
            print(1)
            #d_lines.append(np.nan)
            #print(l)
    
    d_lawyer_names = []
    defendants = []
    for item in d_lines:
        if(lawyer.search(item)):
            d_lawyer_tokens = item.split()
            for name in d_lawyer_tokens:
                if(lawyer.search(name)):
                    name_idx = d_lawyer_tokens.index(name)
                    try: d_lawyer_names.append(d_lawyer_tokens[name_idx + 1].strip())
                    except: d_lawyer_names.append(np.nan)
        else:
            defendants.append(item)
    if(len(defendants) == 0):
        #print(d_lines)
        defendants.append(np.nan)
    
    return [d_lawyer_names, defendants]
            

### Parsing cases

In [10]:
cases_dict = dict()
case_n = 0
for case in c_dat:
    case_n = case_n+1
    cases_dict[case_n]= {}
    d = cases_dict[case_n]
    zero_response = True
    for line in case:
        if(head_judge_fr.match(line) or head_judge_it.match(line)):
            date_and_lang = date_and_lang_fun(line)
            d['head judge'] = head_judge_fun(line)
            d['day'] = date_and_lang[0]
            d['month'] = date_and_lang[1]
            d['year'] = date_and_lang[2]
            d['language'] = date_and_lang[3]
        if((parties_fr.match(line) or parties_it.match(line)) and zero_response):
            parties = parties_fun(line)            
            d['plaintiff lawyer'] = parties[0][0]
            d['plaintiffs'] = parties[0][1]
            d['defendant lawyer'] = parties[1][0]
            d['defendants'] = parties[1][1]
            zero_response = False

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
['111', 'Jnin', '1899']
['111', 'Jnin', '1899']
['16', 'Déceinbre', '1903']
['16', 'Déceinbre', '1903']
1


In [11]:
cases_dict[23]

{'day': nan,
 'head judge': 'GIACCONE.',
 'language': '',
 'month': nan,
 'year': nan}

In [12]:
cases_dict[10]

{'day': 21,
 'defendant lawyer': ['CARITATO.'],
 'defendants': ['LA Coun ;',
  'Attendu que si avant la publication',
  'des nouveaux Codes, le droitMusulman',
  'ne permettait pas la cession des crè-'],
 'head judge': 'LAPEN N A',
 'language': 'italian',
 'month': 11,
 'plaintiff lawyer': ['DE'],
 'plaintiffs': ['sement demeuraient soumises a la',
  'prescription de 15 ans,’',
  ['Julien', 'J']],
 'year': 1878}

In [15]:
a = [item.rstrip() for item in c_dat[4559-1]]
a = ' '.join(a)
a = re.sub(r'\-\s', '', a)
print(a)

Loi de liquidation; art. 86; application. L'article 80 de la loi de liquidation ne distingue pas entre les obligations pouvant résulter d'un contrat et les droits prenant leur source dans toute autre cause; il est général et absolu, et il importe peu que la somme réclamée constitue un prix de vente. S. A. Ismaïl Pacha Av. Zaja contre Gouvernement Egyptien et consorts Av. Schiarabati. La Cour, Attendu qu'aux termes de l'art. 86 de la loi de liquidation du 17 Juillet 1880, toute action quelconque intentée contre le Gouvernement ou une Administration de l'Etat, à partir de la publication de la dite loi et à raison de droits acquis antérieurement au 1" Janvier 1880, est déclarée irrecevable; Que cet article ne distingue pas entre les obligations pouvant résulter d'un contrat et les droits prenant leur source dans toute autre cause; qu'il est général et absolu; qu'il importe donc peu au procès actuel que la somme réclamée constitue un prix de vente; Que l'article ne comporte que deux except

In [16]:
cases_dict_df = pd.DataFrame(cases_dict).transpose()
os.chdir('/Users/simabiondi/Desktop')
cases_dict_df.to_csv("thesis_dict_data_admin_gov.csv", header = True)

In [20]:
cases_dict_df.size

42183

In [222]:
c_dat[1521]

['l. - Contravention , chef de poste\n',
 'de police; procès-verbal; force\n',
 'probante.\n',
 'Il. - Maison de tolérance; consta-\n',
 'tation; base; jugement; effets.\n',
 ' I. - Le procès-verbal de contraven-\n',
 'tion dressé non par un simple agent,\n',
 'mais par un ofﬁcier de police judiciaire,\n',
 "tel qu'un chef de poste de police (ma-\n",
 'mour lcism), fait foi de son contenu jus-\n',
 "qu'd preuve du contraire, sans qu'il ait\n",
 'encore d être afﬁrmé sous la foi du\n',
 'serment.\n',
 'II. - Est déﬁnitif et ‘souveraine-\n',
 "ment rendu sur le fait qu'il constate, le\n",
 "jugement qui, pour déclarer que l'éta-\n",
 "blissement tenu par l'inculpé est une\n",
 "maison de tolérance, s'est basé sur un\n",
 "arrêté de l'autorité administrative, pris\n",
 "du consentement de l'autorité consulaire\n",
 'compétente, arrêté et consentement dal-\n',
 "ment notiﬁés d l'intéressé.\n",
 '\n',
 'MAssAoun et. BAz\n',
 'contre\n',
 'MINISTÈRE PUBLIC.\n',
 'LA COUR,\n',
 "Attendu que l

In [115]:
[print(lindex,c_dat[1522][lindex],line) for lindex,line in enumerate(c_dat[1522]) if "Le Président," in line]
c_dat[1522]
#month[22]
#language[22]

225 Le Président,
 Le Président,

237 Le Président,
 Le Président,



['Droit musulman; succession ;\n',
 'dettes; biens successoraux;\n',
 'vente; autorité compétente; li-\n',
 'quidateur; acquéreur; paye-\n',
 'ment du prix; créancier hypo-\n',
 'thécaire; créancier ordinaire;\n',
 'affectation hypothécaire; nul-\n',
 'lité.\n',
 'En droit musulman, les biens du\n',
 'défunt sont séparés des biens de ses\n',
 'héritiers et ccuæ-ci ne peuvent pas\n',
 'vendre les biens de la succession avant\n',
 'le payement de toutes les dettes; mais\n',
 "la vente faite par I'iHtcrnzédiaz're de\n",
 "l'autorité compétente et par un liqui-\n",
 'dateur nommé d cet e/ﬂv, pour payer\n',
 'les dettes hypothécaires ou autres et\n',
 'liquider la succession, est valable et\n',
 'opposable si tous créctnciers de la suc-\n',
 "cession n'ayant pas acquis antérieure-\n",
 "ment un droit réel sur l'immeuble\n",
 'vendu.\n',
 'J\n',
 'L affectation hg/potleétraire prise et le\n',
 'commandement immobilier inscrit par\n',
 'un de ces créanciers postérieurement d\n',
 "la trente s

In [ ]:
if(month == np.nan)

In [20]:
cases_df.iloc[100,:]

NameError: name 'cases_df' is not defined

In [11]:
(month[22])

nan

### Creating the dataframe from the metadata

In [12]:
d_zip = list(zip(plaintiff, p_lawyer, defendant, d_lawyer, date, day, month, year, head_judge, 
                 language, original_city, verdict_lines))
cases_df = pd.DataFrame(d_zip, columns = ['plaintiff', 'plaintiff_lawyer', 'defendant', 
                                          'defendant_lawyer', 'date', 'day', 
                                          'month', 'year', 'head judge', 'language', 'original city',
                                         'verdict'])


In [11]:
cases_df

,plaintiff,plaintiff_lawyer,defendant,defendant_lawyer,date,day,month,year,head judge,language,original city,verdict
0,[],[],[],[Dmmnm.],NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
1,[],[],"[Fratelli Kindineko Av. Dmmnm., Considerato ch...",[A.],1878-11-07,7.0,11.0,1878.0,LAPENNA.,italian,NaN,[]
2,[],[],"[‘Le Gouvernement. Egyptieu Av. A. CoLnccr., i...",[m:],1878-11-07,7.0,11.0,1878.0,LAPENNA,italian,NaN,[]
3,[],[],"[Julien Josseraiid Av. m: LA POMMERAYE., LA Co...",[DE],NaN,NaN,NaN,NaN,LAPENNA,NaN,NaN,[]
4,[],[],"[Th. Zigada Av. DE LA Pommn nm., La COURI, Ouï...",[GARGANO.],NaN,NaN,NaN,NaN,NaN,NaN,NaN,[]
5,[],[],[Dame Emilie Morpurgo et consorts Av. GARGANO....,[CARITATO.],1818-11-21,21.0,11.0,1818.0,LAPENN A,italian,NaN,[]
6,[],[NINCI],"[Hamin Abdel Ahmed Saha Av. CARITATO., LA Coun...",[61mm).],1878-11-21,21.0,11.0,1878.0,LAPEN N A,italian,NaN,[]
7,"[Civ.);, S. E. Ahmed Pacha Daramalli et consorts]",[],[Elia Septon],[Ennum.],1878-11-28,28.0,11.0,1878.0,,italian,NaN,[]
8,[],[],[Moise Natalia],[ZAYA.],1878-11-28,28.0,11.0,1878.0,LAPEN NA,italian,NaN,[]
9,[],[],[llarotti Sigismond],[A.],1878-12-04,4.0,12.0,1878.0,LAPE1\ NA,french,NaN,[]


### Downloading to CSV

In [60]:
cases_df.to_csv("thesis_data1.csv", header = True)

In [43]:
c_dat[]

plaintiff                                                          []
plaintiff_lawyer                                                   []
defendant           [‘Le Gouvernement. Egyptieu Av. A. CoLnccr., i...
defendant_lawyer                                                 [m:]
date                                                       1878-11-07
day                                                                 7
month                                                              11
year                                                             1878
head judge                                                    LAPENNA
language                                                      italian
original city                                                     NaN
verdict                                                            []
Name: 2, dtype: object

In [61]:
c_dat[21]

["Douanes. - Saisie de tabacs dans le domicile d'un\n",
 'particulier. - Rovendicatiom- Propriété.\n',
 'Les employés des douanes ont le\n',
 'droit de sïntroduire dans le domicile\n',
 'd’un particulier, avec son autorisa-\n',
 'tion, pour y saisir des tabacs présu-\n',
 'més de contrebande qui y ont été dé-\n',
 'posés à son insu par des inconnus.\n',
 'Dans tous les cas, celui qui reven-\n',
 'dique en pareil cas les tabacs saisis,\n',
 'doit prouver, avant tout, quïl en était\n',
 "propriétaire ,'\n",
 'Giuseppe Spagnol Av. SEMIANI\n',
 'contre\n',
 'Le Gouvernement Egyptien Av. A. CoLUeci\n',
 'LA COUR,\n',
 '_ Ouï les parties en leurs dires et con-\n',
 'clusions;\n',
 "70 COUR D'APPEL\n",
 "coma D'APPELLO\n",
 '\n',
 'Le Ministère Public entendu;\n',
 'Après en avoir délibéré conformément\n',
 'à la loi;\n',
 'Attendu que le sieur Spagnola assi-\n',
 'gne Le GouvernementEgyptien devant\n',
 "le ’l‘ribunal Civil d’ Alexandrie à l’eti'et\n",
 'de s’entendre condamner à Pimmédiate\

### Scraps

In [122]:
#cases_df.iloc[29,:]
#R
##%load_ext rpy2.ipython
##a  = pandas2ri.py2ri(cases_df)
##%%R -i cases_df_no_date